In [ ]:
# ================================================================
# LICITA+ — Análise de Desempenho dos Modelos de Linguagem
# VERSÃO FINAL: Gráficos Alinhados aos Objetivos do TCC
# ================================================================

import subprocess, sys, warnings, os, base64
for pkg in ["pandas>=2.0.0","numpy>=1.24.0","matplotlib>=3.7.0","scipy>=1.10.0",
            "openpyxl>=3.1.0","seaborn>=0.12.0"]:
    subprocess.check_call([sys.executable,"-m","pip","install","-q",pkg])
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuração estética (ABNT-friendly)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 12

# ========== CONFIGURAÇÃO ==========
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    BASE_DIR = "/content/drive"
except:
    BASE_DIR = "/content"

PASTA_DRIVE = "My Drive/📁 FACULDADE/2025 - 8º Período/Trabalho de Conclusão de Curso II"
NOME_ARQUIVO = "relatorio_final_20251016_023256.xlsx"
CAMINHO = f"{BASE_DIR}/{PASTA_DRIVE}/{NOME_ARQUIVO}"

TS = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = f"{BASE_DIR}/{PASTA_DRIVE}/analise_licita_{TS}"
os.makedirs(OUT_DIR, exist_ok=True)
IMG_DIR = f"{OUT_DIR}/graficos"
os.makedirs(IMG_DIR, exist_ok=True)

print("="*80)
print("LICITA+ — ANÁLISE COMPLETA DE DESEMPENHO")
print("7 Gráficos + Tabelas Detalhadas")
print(f"Execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("="*80)

# ========== LEITURA DOS DADOS ==========
print("\n[1] Carregando dataset...")
df = pd.read_excel(CAMINHO, engine="openpyxl")
print(f"✓ Total de registros: {len(df)}")

REF_MANUAL = 'preco_manual_med'
LICITADO = 'valor_unitario_licitado'

MODELOS = {
    'Modelo 01 (GPT-4o-mini)': {
        'produto': 'produto_identificado_modelo_01',
        'preco': 'preco_estimado_modelo_01'
    },
    'Modelo 02 (Claude Sonnet 4.5)': {
        'produto': 'produto_identificado_modelo_02',
        'preco': 'preco_estimado_modelo_02'
    },
    'Modelo 03 (DeepSeek-V2)': {
        'produto': 'produto_identificado_modelo_03',
        'preco': 'preco_estimado_modelo_03'
    }
}

# ========== FUNÇÕES DE CÁLCULO ==========
def calcular_mae(y_true, y_pred):
    mask = (~pd.isna(y_true)) & (~pd.isna(y_pred))
    if mask.sum() == 0:
        return np.nan
    return np.abs(y_true[mask] - y_pred[mask]).mean()

def calcular_mape(y_true, y_pred):
    mask = (~pd.isna(y_true)) & (~pd.isna(y_pred)) & (y_true != 0)
    if mask.sum() == 0:
        return np.nan
    return (np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]).mean() * 100)

def calcular_ape_series(y_true, y_pred):
    """Retorna série de APE para análises detalhadas"""
    mask = (~pd.isna(y_true)) & (~pd.isna(y_pred)) & (y_true != 0)
    ape = pd.Series(index=y_true.index, dtype=float)
    ape[mask] = np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]) * 100
    return ape

def calcular_taxa_identificacao(df, col_produto):
    """Calcula % de produtos identificados (campo preenchido)"""
    total = len(df)
    identificados = df[col_produto].notna().sum()
    return (identificados / total * 100) if total > 0 else 0

print("\n[2] Calculando métricas gerais...")

# Categorias
categorias = ['Modelo 01', 'Modelo 02', 'Modelo 03']
categorias_completo = ['Valor Licitado'] + categorias

# Métricas gerais
mae_valores = []
mape_valores = []
taxa_identificacao = []

# Licitado
mae_valores.append(calcular_mae(df[REF_MANUAL], df[LICITADO]))
mape_valores.append(calcular_mape(df[REF_MANUAL], df[LICITADO]))
taxa_identificacao.append(100.0)  # Sempre disponível

# Modelos
for nome, cols in MODELOS.items():
    mae_valores.append(calcular_mae(df[REF_MANUAL], df[cols['preco']]))
    mape_valores.append(calcular_mape(df[REF_MANUAL], df[cols['preco']]))
    taxa_identificacao.append(calcular_taxa_identificacao(df, cols['produto']))

# Calcular APE para todos
df['ape_licitado'] = calcular_ape_series(df[REF_MANUAL], df[LICITADO])
for idx, (nome, cols) in enumerate(MODELOS.items()):
    df[f'ape_modelo_{idx+1}'] = calcular_ape_series(df[REF_MANUAL], df[cols['preco']])

# ========== CLASSIFICAÇÃO POR FAIXAS ==========
def classificar_faixa_grossa(valor):
    if pd.isna(valor):
        return 'Indefinido'
    elif valor <= 1000:
        return '0-1000'
    elif valor <= 3000:
        return '1000-3000'
    else:
        return '>3000'

def classificar_faixa_detalhada(valor):
    if pd.isna(valor):
        return 'Indefinido'
    elif valor < 50:
        return '<50'
    elif valor <= 200:
        return '50-200'
    elif valor <= 500:
        return '200-500'
    elif valor <= 1000:
        return '500-1000'
    elif valor <= 3000:
        return '1000-3000'
    else:
        return '>3000'

df['faixa_grossa'] = df[REF_MANUAL].apply(classificar_faixa_grossa)
df['faixa_detalhada'] = df[REF_MANUAL].apply(classificar_faixa_detalhada)

faixas_grossas = ['0-1000', '1000-3000', '>3000']
faixas_detalhadas = ['<50', '50-200', '200-500', '500-1000', '1000-3000', '>3000']

print("\n[3] Calculando métricas por faixa...")

# Preparar dados para Fig 3 e Fig 7
dados_evolucao = {fx: [] for fx in faixas_detalhadas}

for faixa in faixas_detalhadas:
    subset = df[df['faixa_detalhada'] == faixa]
    if len(subset) == 0:
        for cat in categorias_completo:
            dados_evolucao[faixa].append(np.nan)
        continue

    # Licitado
    dados_evolucao[faixa].append(calcular_mape(subset[REF_MANUAL], subset[LICITADO]))

    # Modelos
    for idx, (nome, cols) in enumerate(MODELOS.items()):
        dados_evolucao[faixa].append(calcular_mape(subset[REF_MANUAL], subset[cols['preco']]))

# Preparar dados para Fig 6 (classificação por nível de erro)
def classificar_por_nivel_erro(ape_series):
    """Classifica cada item em: Excelente, Bom, Razoável, Impreciso"""
    categorias = []
    for ape in ape_series:
        if pd.isna(ape):
            categorias.append('Sem dados')
        elif ape < 10:
            categorias.append('Excelente (<10%)')
        elif ape <= 30:
            categorias.append('Bom/Razoável (10-30%)')
        elif ape <= 50:
            categorias.append('Impreciso (30-50%)')
        else:
            categorias.append('Muito Impreciso (>50%)')
    return categorias

classificacao_erro = {}
classificacao_erro['Licitado'] = classificar_por_nivel_erro(df['ape_licitado'])
for idx in range(3):
    classificacao_erro[f'Modelo 0{idx+1}'] = classificar_por_nivel_erro(df[f'ape_modelo_{idx+1}'])

# ========== TABELAS ==========
print("\n[4] Gerando tabelas...")

# Tabela 1: Resumo Geral
df_resumo = pd.DataFrame({
    'Abordagem': categorias_completo,
    'MAE (R$)': mae_valores,
    'MAPE (%)': mape_valores,
    'Taxa Identificação (%)': taxa_identificacao
})
df_resumo.to_excel(f"{OUT_DIR}/tabela1_resumo_geral.xlsx", index=False)
print(df_resumo.to_string(index=False))

# Tabela 2: MAPE por Faixa Grossa
dados_faixa_grossa = []
for faixa in faixas_grossas:
    subset = df[df['faixa_grossa'] == faixa]
    if len(subset) == 0:
        continue

    row = {'Faixa': faixa, 'N': len(subset)}

    # Licitado
    row['MAPE Licitado (%)'] = calcular_mape(subset[REF_MANUAL], subset[LICITADO])

    # Modelos
    for idx, (nome, cols) in enumerate(MODELOS.items()):
        row[f'MAPE Modelo 0{idx+1} (%)'] = calcular_mape(subset[REF_MANUAL], subset[cols['preco']])

    dados_faixa_grossa.append(row)

df_faixa_grossa = pd.DataFrame(dados_faixa_grossa)
df_faixa_grossa.to_excel(f"{OUT_DIR}/tabela2_mape_faixa_grossa.xlsx", index=False)

# Tabela 3: Desempenho por Faixa Detalhada
dados_faixa_det = []
for faixa in faixas_detalhadas:
    subset = df[df['faixa_detalhada'] == faixa]
    if len(subset) == 0:
        continue

    for idx_cat, cat in enumerate(categorias_completo):
        if cat == 'Valor Licitado':
            col_preco = LICITADO
        else:
            col_preco = list(MODELOS.values())[idx_cat-1]['preco']

        dados_faixa_det.append({
            'Faixa': faixa,
            'Abordagem': cat,
            'N': len(subset),
            'MAPE (%)': calcular_mape(subset[REF_MANUAL], subset[col_preco]),
            'MAE (R$)': calcular_mae(subset[REF_MANUAL], subset[col_preco])
        })

df_faixa_det = pd.DataFrame(dados_faixa_det)
df_faixa_det.to_excel(f"{OUT_DIR}/tabela3_desempenho_detalhado.xlsx", index=False)

print(f"\n✓ 3 tabelas Excel salvas em {OUT_DIR}")

# ========== CORES ==========
CORES = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
CORES_NIVEL = ['#27ae60', '#f39c12', '#e67e22', '#c0392b', '#95a5a6']

print("\n[5] Gerando 7 gráficos")

# ============================================================
# GRÁFICO 1: MAE por Modelo
# ============================================================
fig1, ax1 = plt.subplots(figsize=(10, 6))

bars = ax1.bar(range(len(categorias_completo)), mae_valores, color=CORES,
               edgecolor='black', linewidth=1.2, width=0.6)

for i, (bar, valor) in enumerate(zip(bars, mae_valores)):
    ax1.text(bar.get_x() + bar.get_width()/2, valor + 30,
             f'R$ {valor:.2f}', ha='center', va='bottom',
             fontweight='bold', fontsize=10)

ax1.set_ylabel('MAE (R$)', fontweight='bold')
ax1.set_title('Figura 2 — Erro Absoluto Médio (MAE) por Abordagem',
              fontweight='bold', pad=15)
ax1.set_xticks(range(len(categorias_completo)))
ax1.set_xticklabels(categorias_completo, fontsize=10)
ax1.grid(axis='y', alpha=0.3, linestyle=':')
ax1.set_ylim(0, max(mae_valores) * 1.2)

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura1_mae.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 2 - MAE")

# ============================================================
# GRÁFICO 2: MAPE por Modelo
# ============================================================
fig2, ax2 = plt.subplots(figsize=(10, 6))

bars = ax2.bar(range(len(categorias_completo)), mape_valores, color=CORES,
               edgecolor='black', linewidth=1.2, width=0.6)

ax2.axhline(y=30, color='red', linestyle='--', linewidth=2,
            label='Limiar Aceitável (30%)', alpha=0.7)

for i, (bar, valor) in enumerate(zip(bars, mape_valores)):
    ax2.text(bar.get_x() + bar.get_width()/2, valor + 3,
             f'{valor:.1f}%', ha='center', va='bottom',
             fontweight='bold', fontsize=10)

ax2.set_ylabel('MAPE (%)', fontweight='bold')
ax2.set_title('Figura 3 — Erro Percentual Absoluto Médio (MAPE) por Abordagem',
              fontweight='bold', pad=15)
ax2.set_xticks(range(len(categorias_completo)))
ax2.set_xticklabels(categorias_completo, fontsize=10)
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3, linestyle=':')
ax2.set_ylim(0, max(mape_valores) * 1.2)

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura2_mape.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 3 - MAPE")

# ============================================================
# GRÁFICO 3: Evolução do MAPE por Faixa (LINHA)
# ============================================================
fig3, ax3 = plt.subplots(figsize=(12, 6))

x_pos = range(len(faixas_detalhadas))

# Plotar linhas
for idx, cat in enumerate(categorias_completo):
    valores = [dados_evolucao[fx][idx] for fx in faixas_detalhadas]
    ax3.plot(x_pos, valores, marker='o', linewidth=2.5, markersize=8,
             label=cat, color=CORES[idx], alpha=0.8)

ax3.axhline(y=30, color='red', linestyle='--', linewidth=2,
            label='Limiar Aceitável (30%)', alpha=0.5)

ax3.set_xlabel('Faixa de Preço (R$)', fontweight='bold', fontsize=11)
ax3.set_ylabel('MAPE (%)', fontweight='bold', fontsize=11)
ax3.set_title('Figura 4 — Evolução do MAPE por Faixa de Preço',
              fontweight='bold', fontsize=12, pad=15)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(faixas_detalhadas, fontsize=9, rotation=20, ha='right')
ax3.legend(fontsize=9, loc='upper right')
ax3.grid(True, alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura3_evolucao_mape.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 3 - Evolução MAPE (linha)")

# ============================================================
# GRÁFICO 4: Boxplot de Erros Percentuais (APE)
# ============================================================
fig4, ax4 = plt.subplots(figsize=(12, 6))

dados_boxplot = []
labels_boxplot = []

dados_boxplot.append(df['ape_licitado'].dropna())
labels_boxplot.append('Valor\nLicitado')

for idx in range(3):
    dados_boxplot.append(df[f'ape_modelo_{idx+1}'].dropna())
    labels_boxplot.append(f'Modelo\n0{idx+1}')

bp = ax4.boxplot(dados_boxplot, labels=labels_boxplot, patch_artist=True,
                 showmeans=True, meanprops=dict(marker='D', markerfacecolor='red', markersize=8))

# Colorir caixas
for patch, cor in zip(bp['boxes'], CORES):
    patch.set_facecolor(cor)
    patch.set_alpha(0.6)

ax4.axhline(y=30, color='orange', linestyle='--', linewidth=2,
            label='Limiar 30%', alpha=0.7)
ax4.axhline(y=0, color='green', linestyle='-', linewidth=1.5, alpha=0.5)

ax4.set_ylabel('APE - Erro Percentual Absoluto (%)', fontweight='bold', fontsize=11)
ax4.set_title('Figura 4 — Distribuição dos Erros Percentuais por Abordagem',
              fontweight='bold', fontsize=12, pad=15)
ax4.legend(fontsize=9)
ax4.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura4_boxplot_ape.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 4 - Boxplot APE")

# ============================================================
# GRÁFICO 5: MAPE por Faixa Grossa (Colunas Agrupadas)
# ============================================================
fig5, ax5 = plt.subplots(figsize=(12, 6))

x = np.arange(len(faixas_grossas))
width = 0.2

for idx, cat in enumerate(categorias_completo):
    valores = []
    for faixa in faixas_grossas:
        subset = df[df['faixa_grossa'] == faixa]
        if cat == 'Valor Licitado':
            col = LICITADO
        else:
            col = list(MODELOS.values())[idx-1]['preco']
        valores.append(calcular_mape(subset[REF_MANUAL], subset[col]))

    bars = ax5.bar(x + idx*width, valores, width, label=cat,
                   color=CORES[idx], edgecolor='black', linewidth=1)

    for j, (bar, val) in enumerate(zip(bars, valores)):
        if not pd.isna(val):
            ax5.text(bar.get_x() + bar.get_width()/2, val + 3,
                     f'{val:.1f}%', ha='center', va='bottom',
                     fontsize=8, fontweight='bold')

ax5.axhline(y=30, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax5.set_xlabel('Faixa de Preço (R$)', fontweight='bold', fontsize=11)
ax5.set_ylabel('MAPE (%)', fontweight='bold', fontsize=11)
ax5.set_title('Figura 5 — MAPE por Faixa de Preço (0-1000, 1000-3000, >3000)',
              fontweight='bold', fontsize=12, pad=15)
ax5.set_xticks(x + width * 1.5)
ax5.set_xticklabels(faixas_grossas, fontsize=10)
ax5.legend(fontsize=9, loc='upper right')
ax5.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura5_mape_faixa_grossa.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 5 - MAPE Faixa Grossa")

# ============================================================
# GRÁFICO 6: Classificação por Nível de Erro (Barras Empilhadas)
# ============================================================
fig6, ax6 = plt.subplots(figsize=(12, 7))

niveis = ['Excelente (<10%)', 'Bom/Razoável (10-30%)', 'Impreciso (30-50%)',
          'Muito Impreciso (>50%)', 'Sem dados']

dados_empilhados = []
for cat in ['Licitado', 'Modelo 01', 'Modelo 02', 'Modelo 03']:
    contagens = pd.Series(classificacao_erro[cat]).value_counts()
    percentuais = (contagens / len(df) * 100).reindex(niveis, fill_value=0)
    dados_empilhados.append(percentuais.values)

dados_empilhados = np.array(dados_empilhados).T

x_pos = range(len(categorias_completo))
bottom = np.zeros(len(categorias_completo))

for idx, nivel in enumerate(niveis):
    bars = ax6.bar(x_pos, dados_empilhados[idx], bottom=bottom,
                   label=nivel, color=CORES_NIVEL[idx], edgecolor='white', linewidth=1)

    # Adicionar percentuais nas barras (só se > 5%)
    for i, val in enumerate(dados_empilhados[idx]):
        if val > 5:
            ax6.text(i, bottom[i] + val/2, f'{val:.1f}%',
                     ha='center', va='center', fontsize=8, fontweight='bold', color='white')

    bottom += dados_empilhados[idx]

ax6.set_ylabel('Percentual de Itens (%)', fontweight='bold', fontsize=11)
ax6.set_title('Figura 6 — Classificação dos Itens por Nível de Erro (APE)',
              fontweight='bold', fontsize=12, pad=15)
ax6.set_xticks(x_pos)
ax6.set_xticklabels(categorias_completo, fontsize=10)
ax6.set_ylim(0, 100)
ax6.legend(fontsize=9, loc='upper left', bbox_to_anchor=(1, 1))
ax6.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura6_classificacao_erro.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 6 - Classificação por Nível de Erro")

# ============================================================
# GRÁFICO 7: MAPE por Faixa Detalhada (Colunas Agrupadas)
# ============================================================
fig7, ax7 = plt.subplots(figsize=(14, 7))

x = np.arange(len(faixas_detalhadas))
width = 0.2

for idx, cat in enumerate(categorias_completo):
    valores = []
    for faixa in faixas_detalhadas:
        subset = df[df['faixa_detalhada'] == faixa]
        if cat == 'Valor Licitado':
            col = LICITADO
        else:
            col = list(MODELOS.values())[idx-1]['preco']
        valores.append(calcular_mape(subset[REF_MANUAL], subset[col]))

    bars = ax7.bar(x + idx*width, valores, width, label=cat,
                   color=CORES[idx], edgecolor='black', linewidth=1)

    for j, (bar, val) in enumerate(zip(bars, valores)):
        if not pd.isna(val):
            ax7.text(bar.get_x() + bar.get_width()/2, val + 5,
                     f'{val:.0f}%', ha='center', va='bottom',
                     fontsize=7, fontweight='bold')

ax7.axhline(y=30, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax7.set_xlabel('Faixa de Preço (R$)', fontweight='bold', fontsize=11)
ax7.set_ylabel('MAPE (%)', fontweight='bold', fontsize=11)
ax7.set_title('Figura 7 — MAPE por Faixa Detalhada (<50, 50-200, ..., >3000)',
              fontweight='bold', fontsize=12, pad=15)
ax7.set_xticks(x + width * 1.5)
ax7.set_xticklabels(faixas_detalhadas, fontsize=9)
ax7.legend(fontsize=9, loc='upper right')
ax7.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig(f"{IMG_DIR}/figura7_mape_faixa_detalhada.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ Figura 7 - MAPE Faixa Detalhada")

print(f"\n✓ 7 gráficos salvos em {IMG_DIR}")

# ========== RELATÓRIO HTML ==========
print("\n[6] Gerando relatório HTML...")

def img_to_base64(caminho):
    with open(caminho, "rb") as f:
        return "data:image/png;base64," + base64.b64encode(f.read()).decode()

html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>LICITA+ — Relatório Final</title>
    <style>
        body {{
            font-family: 'Times New Roman', serif;
            margin: 0;
            padding: 40px;
            line-height: 1.8;
            color: #2c3e50;
            max-width: 1400px;
            margin: 0 auto;
        }}
        h1 {{
            text-align: center;
            color: #2c3e50;
            border-bottom: 4px solid #3498db;
            padding-bottom: 15px;
            margin-bottom: 30px;
        }}
        h2 {{
            color: #34495e;
            margin-top: 50px;
            border-bottom: 2px solid #3498db;
            padding-bottom: 8px;
        }}
        .destaque {{
            background-color: #fff9e6;
            padding: 20px;
            border-left: 5px solid #f39c12;
            margin: 30px 0;
            border-radius: 5px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 25px 0;
            font-size: 11px;
        }}
        th, td {{
            border: 1px solid #bdc3c7;
            padding: 10px;
            text-align: center;
        }}
        th {{
            background-color: #3498db;
            color: white;
            font-weight: bold;
        }}
        tr:nth-child(even) {{
            background-color: #ecf0f1;
        }}
        img {{
            max-width: 100%;
            height: auto;
            display: block;
            margin: 30px auto;
            border: 2px solid #bdc3c7;
            padding: 10px;
            background: white;
        }}
        .rodape {{
            margin-top: 60px;
            text-align: center;
            font-size: 10px;
            color: #7f8c8d;
            border-top: 2px solid #bdc3c7;
            padding-top: 20px;
        }}
    </style>
</head>
<body>
    <h1>LICITA+ — Relatório de Análise Completo</h1>

    <div class="destaque">
        <strong>Sistema de Análise de Licitações com Grandes Modelos de Linguagem</strong><br><br>
        Avaliação de desempenho de três LLMs (GPT-4o-mini, Claude Sonnet 4.5, DeepSeek-V2)
        na estimativa de preços de itens licitados.<br><br>
        <strong>Métricas:</strong> MAE (R$), MAPE (%), Taxa de Identificação (%)<br>
        <strong>Data:</strong> {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}<br>
        <strong>Total de Itens:</strong> {len(df)}
    </div>

    <h2>1. Resumo Geral de Desempenho</h2>
    {df_resumo.to_html(index=False, border=0)}

    <h2>2. Análise de Erros</h2>

    <h3>2.1 Erro Absoluto Médio (MAE)</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura1_mae.png')}" alt="Figura 1">

    <h3>2.2 Erro Percentual Médio (MAPE)</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura2_mape.png')}" alt="Figura 2">

    <h3>2.3 Distribuição dos Erros Percentuais</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura4_boxplot_ape.png')}" alt="Figura 4">

    <h3>2.4 Classificação por Nível de Erro</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura6_classificacao_erro.png')}" alt="Figura 6">

    <h2>3. Análise por Faixa de Preço</h2>

    <h3>3.1 Evolução do MAPE por Faixa</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura3_evolucao_mape.png')}" alt="Figura 3">

    <h3>3.2 MAPE por Faixa Grossa</h3>
    {df_faixa_grossa.to_html(index=False, border=0)}
    <img src="{img_to_base64(f'{IMG_DIR}/figura5_mape_faixa_grossa.png')}" alt="Figura 5">

    <h3>3.3 MAPE por Faixa Detalhada</h3>
    <img src="{img_to_base64(f'{IMG_DIR}/figura7_mape_faixa_detalhada.png')}" alt="Figura 7">

    <div class="rodape">
        <strong>LICITA+</strong> — TCC em Sistemas de Informação — IFPR Palmas<br>
        Luiz Eduardo Zanatta Sendeski e Robson Barbieri Candido<br>
        Orientação: Prof.ª Andreia Marini | Coorientação: Prof. Tarlis Tortelli Portela
    </div>
</body>
</html>
"""

html_path = f"{OUT_DIR}/relatorio_final_licita.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"✓ Relatório HTML: {html_path}")

# ========== SUMÁRIO ==========
print("\n" + "="*80)
print("RESUMO DA EXECUÇÃO")
print("="*80)

melhor_idx = np.argmin(mape_valores[1:]) + 1
melhor_nome = categorias_completo[melhor_idx]

print(f"\n→ Melhor Modelo: {melhor_nome}")
print(f"   MAE: R$ {mae_valores[melhor_idx]:.2f}")
print(f"   MAPE: {mape_valores[melhor_idx]:.2f}%")
print(f"   Taxa Identificação: {taxa_identificacao[melhor_idx]:.2f}%")

print(f"\n→ Arquivos Gerados:")
print(f"   • 7 Gráficos: {IMG_DIR}/figura*.png")
print(f"   • 3 Tabelas: {OUT_DIR}/tabela*.xlsx")
print(f"   • Relatório HTML: {html_path}")

print("\n" + "="*80)
print("ANÁLISE COMPLETA CONCLUÍDA!")
print("="*80)